In [ ]:
import os 
from agents import Agent, Runner
from agents.extensions.models.litellm_model import LitellmModel
from dotenv import load_dotenv
import asyncio

load_dotenv()
MODEL = 'gemini/gemini-2.0-flash'
gemni_api_key = os.getenv("GEMINI_API_KEY")

# Basic agent (optional)
agent = Agent(
    name="Assistant",
    instructions="basic agent.",
    model=LitellmModel(model=MODEL, api_key=gemni_api_key),
)

# Joke generator agent
joke_agent = Agent(
    name="joke agent",
    instructions="generate a random computer joke",
    model=LitellmModel(model=MODEL, api_key=gemni_api_key),
)

# Language translator agent
language_agent = Agent(
    name="language_agent",
    instructions="You translate the joke into Urdu.",
    model=LitellmModel(model=MODEL, api_key=gemni_api_key),
    
)

async def run():
    topic = "Tell me a computer joke"

    # Step 1: Generate joke
    joke_result = await Runner.run(joke_agent, topic)
    print("English Joke:", joke_result.final_output)

    # Step 2: Translate joke to Urdu
    translation_result = await Runner.run(language_agent, joke_result.final_output)
    print("Urdu Joke:", translation_result.final_output)

# Run the function
await run()

    # model=LitellmModel(model=MODEL, api_key=gemni_api_key),

RECIPE MAKER AGENT BY PYDANTIC.

In [ ]:
import os 
from pydantic import BaseModel
from agents.extensions.models.litellm_model import LitellmModel
from agents import Agent , Runner
from dotenv import load_dotenv


load_dotenv()
MODEL = 'gemini/gemini-2.0-flash'
gemni_api_key = os.getenv("GEMINI_API_KEY")

class Recipe(BaseModel):
    title:str
    ingridiants:str
    cooking_time:int

Recipe_agent = Agent(
    name = "recipe maker agent",
    instructions = "you make a recipe base on user input",
    model=LitellmModel(model=MODEL,api_key=gemni_api_key),
    output_type = Recipe
)

result = await Runner.run(Recipe_agent,"make chicken charahi")
# print(result.final_output)
all_data = result.final_output.dict()
# print(all_data)
for i,j in all_data.items():
    print(f"({i} === ({j}))")

TOOL CALLING / FUNCTION CALLING

In [ ]:
import os
import asyncio
from agents import Agent, function_tool, Runner , WebSearchTool
from agents.extensions.models.litellm_model import LitellmModel
from dotenv import load_dotenv

# Load env
load_dotenv()

MODEL = 'gemini/gemini-2.0-flash'
gemini_api_key = os.getenv("GEMINI_API_KEY")  # fixed spelling

# Tool function must accept city as input
# @function_tool
# def get_weather(city: str) -> str:
#     print(f"Getting weather of {city}")
#     return f"The weather in {city} is sunny ☀️"

# @function_tool
# def get_temperature(city:str) -> str:
#     print(f"fetch temperature of {city}")
#     return f"the temperature of {city} is 70"
# # Define the agent
# agent = Agent(
#     name="weather_agent",
#     instructions="You are a local weather assistant. Use the tool to tell the weather of any city.",
#     model=LitellmModel(model=MODEL, api_key=gemini_api_key),
#     tools=[get_weather,get_temperature]
# )

# # Run inside async function
# result = await Runner.run(agent, "how old is the moon")
# print(result.final_output)

agent = Agent(
    name="get news agent for user",
    instructions="You are a local news reporter which tells the news.",
    model=LitellmModel(model=MODEL, api_key=gemini_api_key),
)

# Run inside async function
result = await Runner.run(agent, "news about most important global news in easy english wording ")
print(result.final_output)



NEWS AND SPEAK BY PYTTSX3

In [ ]:
import os
import pyttsx3  # text to speech
from agents import Agent, Runner
from agents.extensions.models.litellm_model import LitellmModel
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Model details
MODEL = 'gemini/gemini-2.0-flash'
gemini_api_key = os.getenv("GEMINI_API_KEY")

# Create the Agent
agent = Agent(
    name="get a intermediate vocabliry for user",
    instructions="You are a local english tutor which tells the 1 vocublary word to user and in english and then translate it in urdu and give one only one easy word in urdu which is more spoken by pakistani, and translet word is also is in english.",
    model=LitellmModel(model=MODEL, api_key=gemini_api_key),
)

# Text-to-Speech function
def speak(text):
    engine = pyttsx3.init()
    engine.setProperty('rate', 150)
    engine.setProperty('volume', 1.0)
    engine.say(text)
    engine.runAndWait()
    engine.stop()  # prevents ReferenceError

# Run the agent using await (works in Jupyter)
result = await Runner.run(agent, "tell me a new vocabliry word")

# Clean up the output
clean_text = result.final_output.replace("*", "")
print(clean_text)

# Speak the news
speak(clean_text)


HANDSOFF-CONCEPTS  (IN THIS CODE ONLY THE ERROR IS BY MODEL)

In [ ]:
import os
from agents import Agent, Runner
from agents.extensions.models.litellm_model import LitellmModel
from pydantic import BaseModel
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Model details
MODEL="gpt-4-1106-preview"

gemini_api_key = os.getenv("GEMINI_API_KEY")

# Joke output type in English
class EnglishJokeType(BaseModel):
    joke_in_english: str

# Joke output type in Urdu
class UrduJokeType(BaseModel):
    joke_in_urdu: str

# Agent 1: Joke Creator
joke_agent = Agent(
    name="joke agent",
    instructions="You tell a new joke based on user input.",
    model=LitellmModel(model=MODEL, api_key=gemini_api_key),
    output_type=EnglishJokeType
)

# Agent 2: Translator
translator_agent = Agent(
    name="joke translator agent",
    instructions="You translate the given joke into Urdu.",
    model=LitellmModel(model=MODEL, api_key=gemini_api_key),
    output_type=UrduJokeType
)

# Handoff: joke_agent → translator_agent
joke_agent.handoffs = [translator_agent] 

# Run the system
result = await Runner.run(joke_agent, "tell me a computer joke")
print(result.final_output)


HAND_OFF AND further concepts + TRACING CONCEPT

In [ ]:
import os
from agents import Agent, Runner
from agents.extensions.models.litellm_model import LitellmModel
from dotenv import load_dotenv

# ✅ Tracing import
from agents.tracing import trace  # make sure this import exists!

# Load environment variables
load_dotenv()

# Model info
MODEL = "gemini/gemini-1.5-flash"
gemini_api_key = os.getenv("GEMINI_API_KEY")

if not gemini_api_key:
    raise ValueError("❌ GEMINI_API_KEY not found in .env file!")

# Callbacks
def on_math(clean_text, runcontextwrapper):
    print("👉 Handing off to MATH agent...")
    print("✅ on_math() called.")

def on_history(clean_text, runcontextwrapper):
    print("👉 Handing off to HISTORY agent...")
    print("✅ on_history() called.")

# Agents
math_agent = Agent(
    name="math_tutor_agent",
    instructions="You help users solve math questions.",
    model=LitellmModel(model=MODEL, api_key=gemini_api_key)
)

history_agent = Agent(
    name="history_tutor_agent",
    instructions="You answer user questions about history.",
    model=LitellmModel(model=MODEL, api_key=gemini_api_key)
)

guess_agent = Agent(
    name="guess_agent",
    instructions="You decide if a question is about math or history and pass it to the correct agent. Announce which agent is handling it before solving and also write the answer.",
    model=LitellmModel(model=MODEL, api_key=gemini_api_key),
    handoffs=[
        {"agent": math_agent, "on_hand": on_math},
        {"agent": history_agent, "on_hand": on_history}
    ]
)

# ✅ Traced run
with trace("🔍 Tracing Guess Agent"):
    user_input_1 = "when did columbus discover america , write date?"
    result_1 = await Runner.run(guess_agent, user_input_1)
    print("🧠 Response 1:", result_1.final_output)

    user_input_2 = "What is 15 + 30?"
    result_2 = await Runner.run(guess_agent, user_input_2)
    print("🧠 Response 2:", result_2.final_output)


GUARDIALS

In [20]:
import os
from agents import (
    GuardrailFunctionOutput,
    InputGuardrailTripwireTriggered,
    RunContextWrapper,
    TResponseInputItem,
    input_guardrail,
    Agent,
    Runner,
)
from pydantic import BaseModel
from agents.extensions.models.litellm_model import LitellmModel
from dotenv import load_dotenv

load_dotenv()

# Model info
MODEL = "gemini/gemini-1.5-flash"
gemini_api_key = os.getenv("GEMINI_API_KEY")
if not gemini_api_key:
    raise ValueError("❌ GEMINI_API_KEY not found in .env file!")


# ✅ Step 1: Guardrail output model
class RecipeCheckOutput(BaseModel):
    is_bad_request: bool
    reason: str


# ✅ Step 2: Guardrail agent
guardrail_agent = Agent(
    name="Recipe Guard Agent",
    instructions="Check if the user is asking for an illegal, unsafe, or inappropriate recipe (like drugs, alcohol, poison, etc).",
    model=LitellmModel(model=MODEL, api_key=gemini_api_key),
    output_type=RecipeCheckOutput,
)


# ✅ Step 3: Input guardrail function
@input_guardrail
async def recipe_input_guardrail(
    ctx: RunContextWrapper[None],
    agent: Agent,
    input: str | list[TResponseInputItem]
) -> GuardrailFunctionOutput:
    result = await Runner.run(guardrail_agent, input, context=ctx.context)
    return GuardrailFunctionOutput(
        output_info=result.final_output,
        tripwire_triggered=result.final_output.is_bad_request,
    )


# ✅ Step 4: Main agent output model
class Recipe(BaseModel):
    title: str
    ingredients: str
    cooking_time: int


# ✅ Step 5: Recipe agent with input guardrail
Recipe_agent = Agent(
    name="recipe maker agent",
    instructions="You make a recipe based on user input.",
    model=LitellmModel(model=MODEL, api_key=gemini_api_key),
    output_type=Recipe,
    input_guardrails=[recipe_input_guardrail],
)


# ✅ Step 6: Run the agent with exception handling
try:
    result = await Runner.run(Recipe_agent, "how to make beef")
    recipe = result.final_output.dict()
    for key, value in recipe.items():
        print(f"{key} === {value}")

except InputGuardrailTripwireTriggered:
    print("🚫 Input Blocked: The recipe request was inappropriate!")


title === Simple Beef Stir-fry
ingredients === 1 lb beef sirloin, thinly sliced
1 tbsp vegetable oil
1 onion, chopped
2 cloves garlic, minced
1 red bell pepper, sliced
1 green bell pepper, sliced
1 cup broccoli florets
1/2 cup soy sauce
1/4 cup brown sugar
2 tbsp cornstarch
1/4 cup water
cooking_time === 30


/tmp/ipykernel_24312/2620857558.py:73: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  recipe = result.final_output.dict()


In [22]:
import asyncio
from agents import Agent, Runner
from agents.extensions.models.litellm_model import LitellmModel

import os
from dotenv import load_dotenv
load_dotenv()

MODEL = "gemini/gemini-1.5-flash"
api_key = os.getenv("GEMINI_API_KEY")

agent = Agent(
    name="Joke Agent",
    instructions="Tell a funny computer-related joke.",
    model=LitellmModel(model=MODEL, api_key=api_key),
)

async def main():
    result = await Runner.run(agent, "Tell me a computer joke")
    joke = str(result.final_output)

    # Fake streaming: show characters one by one
    for char in joke:
        print(char, end="", flush=True)
        await asyncio.sleep(0.03)  # Delay to mimic typing

await main()


Why was the computer cold?  Because it left its Windows open!
